<a href="https://colab.research.google.com/github/tomasonjo/blogs/blob/master/llm/llama_relik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Clone the repository
!git clone https://github.com/richardpaulhudson/coreferee

# Change to the repository directory
%cd coreferee

# Modify the setup.cfg file
!sed -i '39s/spacy>=3.0.0,<3.6.0/spacy>=3.0.0,<3.8.0/' setup.cfg
!sed -i 's/to_version:             3.3.0/to_version:             3.8.0/g' coreferee/lang/en/config.cfg
# Verify the change
!cat setup.cfg | grep spacy

# Install the package
!pip install .

fatal: destination path 'coreferee' already exists and is not an empty directory.
/content/coreferee
keywords= nlp, spacy, spacy-extension, python, machine-learning, coreference, anaphora, coreference-resolution, anaphora-resolution
  spacy>=3.0.0,<3.8.0
spacy_factories =
Processing /content/coreferee
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for coreferee: filename=coreferee-1.4.1-py3-none-any.whl size=182569 sha256=e06422892fedd8078f4ac27fee77e73d974155bfb72aa3f0c037c231e9cb52a9
  Stored in directory: /tmp/pip-ephem-wheel-cache-2sup0d9z/wheels/a3/71/a1/572217811ada49a1a01439dd1902bf93f82f44056909b94138
Successfully built coreferee
  Attempting uninstall: coreferee
    Found existing installation: coreferee 1.4.1
    Uninstalling coreferee-1.4.1:
      Successfully uninstalled coreferee-1.4.1


In [2]:
!pip install --quiet llama-index-extractors-relik llama-index-graph-stores-neo4j llama-index-llms-openai llama-index

In [3]:
!python -m spacy download en_core_web_lg
!python -m coreferee install en

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Processing ./models/en
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for coreferee-model-en: filename=coreferee_model_en-1.0.0-py3-none-any.whl size=65422507 sha256=51a0f26454a45b9f40a91e0b7d3be784e81eaa519a77931196d8035cc0ede18e
  Stored in directory: /tmp/pip-ephem-wheel-cache-5sl944_l/wheels/f6/be/ad/ab1b76b72291d45f7a92a9a29098c985a2fbc9ad9f97c658ac
Successfully built coreferee-model-en
  Attempting uninstall: coreferee-model-en
    Found existing installation: corefer

In [4]:
import nest_asyncio

nest_asyncio.apply()

In [5]:
from llama_index.graph_stores.neo4j import Neo4jPGStore

username="neo4j"
password="rubber-cuffs-radiator"
url="bolt://54.89.19.156:7687"

graph_store = Neo4jPGStore(
    username=username,
    password=password,
    url=url,
    refresh_schema=False
)

In [6]:
import pandas as pd

NUMBER_OF_ARTICLES = 10
news = pd.read_csv("https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/news_articles.csv")
news = news.head(NUMBER_OF_ARTICLES)

In [7]:
import spacy, coreferee

coref_nlp = spacy.load('en_core_web_lg')
coref_nlp.add_pipe('coreferee')


def coref_text(text):
    coref_doc = coref_nlp(text)
    resolved_text = ""

    for token in coref_doc:
        repres = coref_doc._.coref_chains.resolve(token)
        if repres:
            resolved_text += " " + " and ".join(
                [
                    t.text
                    if t.ent_type_ == ""
                    else [e.text for e in coref_doc.ents if t in e][0]
                    for t in repres
                ]
            )
        else:
            resolved_text += " " + token.text

    return resolved_text

print(
    coref_text("Tomaz is so cool. He can solve various Python dependencies and not cry")
)

 Tomaz is so cool . Tomaz can solve various Python dependencies and not cry


In [8]:
from llama_index.core import Document
from datetime import datetime

news["coref_text"] = news["text"].apply(coref_text)
documents = [
    Document(text=f"{row['title']}: {row['coref_text']}") for i, row in news.iterrows()
]


print(coref_text(news['text'][5]))

 Jude Bellingham has also insisted Jude Bellingham move to Real Madrid had nothing to do with money , as Jude Bellingham explained how Jude Bellingham transfer was wrapped up so quickly . 
 “ Money is not a thing for me , ” Jude Bellingham said . 
 “ I do nt think about money at all when I make these kinds of decisions . I never have and I never will . I play the game purely out of love . 
 “ I spoke with people from Real Madrid when I was given permission by Borussia Dortmund and I love the feeling I got from the club . I could n’t hide feeling . I told people straight away what I felt and after that happened on Monday feeling all happened quickly . ” 
 Monday also opened up on how love was given the No.5 shirt , which was worn by defender Jesus Vallejo last season , and what shirt means to Jesus Vallejo to have Zidane ’s old number on Zidane back . 
 “ For a start I ’d like to thank Jesus Jesus Vallejo for letting me wear the No.5 , ” Zidane explained . 
 “ I contacted Jesus Vallejo 

In [9]:
from llama_index.extractors.relik.base import RelikPathExtractor

relik = RelikPathExtractor(
    model="relik-ie/relik-relation-extraction-small", model_config={"skip_metadata": True}
)

# Use on Pro Collab with GPU
# relik = RelikPathExtractor(
#    model="relik-ie/relik-cie-small", model_config={"skip_metadata": True, "device":"cuda"}
# )

                ___              __         
               /\_ \      __    /\ \        
 _ __     __   \//\ \    /\_\   \ \ \/'\    
/\`'__\ /'__`\   \ \ \   \/\ \   \ \ , <    
\ \ \/ /\  __/    \_\ \_  \ \ \   \ \ \\`\  
 \ \_\ \ \____\   /\____\  \ \_\   \ \_\ \_\
  \/_/  \/____/   \/____/   \/_/    \/_/\/_/
                                            
                                            



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[2024-08-08 09:46:54,099] [WARNING] [relik.common.utils.download_from_hf:342] [PID:9250] [RANK:0] Couldn't download index.faiss from relik-ie/encoder-e5-small-v2-wikipedia-relations-index, ignoring


In [10]:
import os

from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI

os.environ["OPENAI_API_KEY"] = "sk-"

llm = OpenAI(model="gpt-4o", temperature=0.0)
embed_model = OpenAIEmbedding(model_name="text-embedding-3-small")

In [11]:
from llama_index.core import PropertyGraphIndex

index = PropertyGraphIndex.from_documents(
    documents,
    kg_extractors=[relik],
    llm=llm,
    embed_model=embed_model,
    property_graph_store=graph_store,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/10 [00:00<?, ?it/s]

Extracting triples:   0%|          | 0/10 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  1.16it/s]
Generating embeddings: 0it [00:00, ?it/s]


In [12]:
query_engine = index.as_query_engine(include_text=True)

response = query_engine.query("What happened at Ryanair?")

print(str(response))

Ryanair dismissed its chief pilot, Aidan Murray, following an internal investigation that revealed a pattern of repeated inappropriate and unacceptable behavior towards female colleagues. The airline took action to terminate Murray's employment immediately, as confirmed by Chief People Officer Darrell Hughes. Murray, who had been with Ryanair for 28 years, is accused of harassing junior colleagues and altering flight rosters to fly with certain female pilots. The dismissal was made public, and Ryanair declined to comment further on individual employee queries.
